In [1]:
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
#path = f"x264/sampledConfigurations_{name_of_technique}_t3.csv"
#lst = ['distBased', 'divDistBased', 'henard', 'random', 'solverBased', 'twise']

path = r"sampledConfigurations_divDistBased_t3.csv"


df = pd.read_csv(path, sep=';')


In [2]:
df

,root,API,OpenCL,CUDA,TextureMemory,Linear1D,Linear2D,Array2D,LocalMemory,Blocksize,...,padding_416,padding_448,padding_480,padding_512,pixelPerThread,pixelPerThread_1,pixelPerThread_2,pixelPerThread_3,pixelPerThread_4,Performance
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,21.178
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,21.185
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,21.188
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,21.256
4,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,21.261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,1,1,0,1,0,0,0,0,1,1,...,0,0,0,0,1,1,0,0,0,106.481
4597,1,1,0,1,0,0,0,0,1,1,...,1,0,0,0,1,1,0,0,0,106.496
4598,1,1,0,1,0,0,0,0,1,1,...,0,0,1,0,1,1,0,0,0,106.592
4599,1,1,1,0,0,0,0,0,1,1,...,0,0,0,0,1,1,0,0,0,112.250


### Geração de feature importance sem a utilização de hiperparâmetros

In [3]:
def run_featureSelection(df, hyperparameters=None, path=None):
    X = df.drop('Performance', axis=1)
    y = df['Performance']

    if hyperparameters is None:
        rf = RandomForestRegressor()
    else:
        rf = RandomForestRegressor(**hyperparameters)
    
    rf.fit(X, y)

    importances = rf.feature_importances_

    feature_importances = pd.DataFrame(importances, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)

    print(feature_importances)
    
    if hyperparameters is None:
        if path is None:
            feature_importances.to_csv("feature_importance.csv", header=True)
        else:
            feature_importances.to_csv("feature_importance_"+path+".csv", header=True)
    else:
        if path is None:
            feature_importances.to_csv("feature_importanceHP.csv", header=True)
        else:
            feature_importances.to_csv("feature_importanceHP_"+path+".csv", header=True)


run_featureSelection(df)

                  importance
LocalMemory         0.469705
bs_1024x1           0.091419
bs_32x1             0.061014
pixelPerThread_1    0.046876
bs_128x8            0.030486
bs_64x16            0.029285
bs_256x4            0.029250
bs_512x2            0.029113
bs_32x32            0.028790
bs_512x1            0.028354
TextureMemory       0.019194
pixelPerThread_2    0.016090
Linear1D            0.014098
OpenCL              0.012298
CUDA                0.009479
pixelPerThread_4    0.008066
bs_256x1            0.006166
Array2D             0.005843
bs_256x2            0.005234
bs_128x4            0.005125
bs_64x8             0.005051
bs_32x16            0.004928
pixelPerThread_3    0.004553
bs_64x1             0.004501
Linear2D            0.004443
bs_32x2             0.004374
bs_64x2             0.004343
bs_32x4             0.003587
Ldg                 0.003241
bs_128x2            0.002910
bs_64x4             0.002821
bs_32x8             0.002798
bs_128x1            0.002188
padding_352   

### Geração de feature importance com a utilização de hiperparâmetros

In [5]:
hyperparams_RF = {
    "max_depth": 25,
    "n_estimators": 10,
    "n_jobs" : -1
}

run_featureSelection(df, hyperparams_RF)

                  importance
LocalMemory         0.469743
bs_1024x1           0.093071
bs_32x1             0.063129
pixelPerThread_1    0.046251
bs_256x4            0.030363
bs_128x8            0.029873
bs_64x16            0.029745
bs_32x32            0.029423
bs_512x2            0.027545
bs_512x1            0.027112
TextureMemory       0.019908
pixelPerThread_2    0.014481
Linear1D            0.013525
CUDA                0.011344
OpenCL              0.010958
pixelPerThread_4    0.007711
bs_256x1            0.005992
pixelPerThread_3    0.005269
bs_128x4            0.005209
bs_32x16            0.005160
bs_256x2            0.005150
Array2D             0.005032
bs_64x1             0.004440
bs_64x8             0.004357
bs_32x2             0.004278
bs_64x2             0.004132
bs_32x4             0.003995
Linear2D            0.003941
bs_64x4             0.003321
Ldg                 0.003223
bs_128x2            0.003220
bs_32x8             0.002995
bs_128x1            0.001914
padding_352   

In [6]:
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

for config_name in samples_config:
    for t in [1,2,3]:
        path = r"sampledConfigurations_"+config_name+"_t"+str(t)+".csv"
        path_name = config_name+"_t"+str(t)
        df = pd.read_csv(path, sep=';')
        
        run_featureSelection(df, None, path_name)
        run_featureSelection(df, hyperparams_RF, path_name)

                    importance
LocalMemory       5.616132e-01
bs_32x1           1.470726e-01
pixelPerThread_1  5.657599e-02
bs_32x8           5.017670e-02
bs_512x1          3.955184e-02
pixelPerThread_4  2.845514e-02
Linear2D          1.444221e-02
bs_64x2           1.291441e-02
bs_32x4           1.139772e-02
bs_32x2           1.044352e-02
bs_128x1          1.003729e-02
bs_64x4           8.104612e-03
bs_128x2          7.594368e-03
CUDA              5.981857e-03
bs_256x2          5.343384e-03
OpenCL            5.075794e-03
padding_512       4.321096e-03
bs_256x1          4.108950e-03
Ldg               3.109878e-03
bs_128x4          2.923278e-03
bs_64x8           2.767475e-03
TextureMemory     2.571288e-03
Linear1D          2.460919e-03
padding_320       2.159126e-03
padding_0         4.887843e-04
padding_480       1.659599e-04
padding_192       7.825687e-05
padding_96        2.756153e-05
pixelPerThread_2  1.863176e-05
pixelPerThread_3  7.441059e-06
padding_448       5.237073e-06
padding_

                  importance
LocalMemory         0.421833
bs_32x1             0.095829
bs_1024x1           0.074171
pixelPerThread_1    0.053509
bs_512x1            0.052222
Linear1D            0.024823
pixelPerThread_2    0.022787
bs_32x32            0.019618
bs_512x2            0.019464
bs_128x8            0.019339
bs_256x4            0.019263
TextureMemory       0.018387
bs_64x16            0.014712
CUDA                0.014457
OpenCL              0.012943
bs_32x2             0.011665
bs_128x1            0.010289
bs_64x1             0.010075
bs_32x8             0.009718
bs_128x2            0.008887
Array2D             0.007966
bs_64x2             0.007773
bs_256x1            0.006972
bs_32x4             0.006875
bs_64x4             0.006429
Linear2D            0.005925
Ldg                 0.005074
pixelPerThread_4    0.003809
pixelPerThread_3    0.003103
padding_352         0.002510
bs_32x16            0.002126
bs_256x2            0.002068
bs_128x4            0.001917
bs_64x8       

                  importance
LocalMemory         0.471958
bs_1024x1           0.090455
bs_32x1             0.064071
pixelPerThread_1    0.048421
bs_256x4            0.030837
bs_128x8            0.029973
bs_64x16            0.028973
bs_512x2            0.028896
bs_32x32            0.028483
bs_512x1            0.027913
TextureMemory       0.019235
pixelPerThread_2    0.014996
Linear1D            0.014072
OpenCL              0.011995
pixelPerThread_4    0.007565
CUDA                0.007258
Array2D             0.007030
bs_256x1            0.006098
bs_256x2            0.005995
bs_32x16            0.005938
bs_128x4            0.005547
bs_64x8             0.005537
pixelPerThread_3    0.005211
Linear2D            0.004230
bs_64x1             0.004158
bs_32x2             0.003816
bs_64x2             0.003398
Ldg                 0.003179
bs_32x4             0.002685
bs_64x4             0.002099
bs_32x8             0.002075
bs_128x1            0.002049
bs_128x2            0.001995
padding_352   

                  importance
LocalMemory         0.374279
bs_32x1             0.111593
bs_1024x1           0.091570
pixelPerThread_1    0.083030
bs_512x1            0.051469
bs_512x2            0.025776
bs_32x32            0.023245
bs_256x4            0.021374
bs_64x16            0.020873
bs_128x8            0.020803
pixelPerThread_2    0.020319
Array2D             0.020265
OpenCL              0.014549
CUDA                0.014095
TextureMemory       0.013470
Linear1D            0.010615
bs_64x1             0.009624
bs_32x2             0.009249
bs_256x1            0.006364
Ldg                 0.005351
bs_32x16            0.004657
bs_32x8             0.004483
bs_32x4             0.004429
bs_128x1            0.004313
bs_128x2            0.004172
bs_64x4             0.004146
bs_64x2             0.004140
bs_64x8             0.003873
bs_256x2            0.003657
pixelPerThread_4    0.003631
bs_128x4            0.003603
Linear2D            0.002598
pixelPerThread_3    0.002575
padding_480   

                  importance
LocalMemory         0.432003
bs_32x1             0.096821
pixelPerThread_1    0.071897
bs_512x1            0.050965
bs_128x8            0.037919
bs_512x2            0.031441
TextureMemory       0.028102
bs_32x32            0.027273
bs_64x16            0.017586
pixelPerThread_2    0.014694
bs_256x4            0.012738
bs_1024x1           0.011925
bs_32x4             0.011658
Linear1D            0.011486
OpenCL              0.010275
bs_32x16            0.009867
CUDA                0.009330
bs_256x2            0.009201
bs_64x8             0.008129
bs_64x1             0.007867
bs_128x4            0.006665
Ldg                 0.006254
bs_64x2             0.006097
pixelPerThread_3    0.006093
bs_64x4             0.005879
bs_256x1            0.005682
bs_32x2             0.005384
bs_128x2            0.005278
pixelPerThread_4    0.004676
padding_352         0.003480
bs_32x8             0.002999
Linear2D            0.002832
bs_128x1            0.002760
Array2D       

                    importance
bs_32x1           3.428525e-01
LocalMemory       3.346091e-01
pixelPerThread_1  1.174495e-01
TextureMemory     2.813605e-02
CUDA              2.512340e-02
bs_256x1          2.369415e-02
bs_512x1          1.725576e-02
pixelPerThread_2  1.707189e-02
OpenCL            1.335387e-02
pixelPerThread_3  1.136401e-02
Linear1D          1.104669e-02
Array2D           1.076341e-02
pixelPerThread_4  6.098914e-03
bs_64x2           5.976935e-03
Linear2D          5.517763e-03
bs_32x8           4.663825e-03
bs_256x2          2.887154e-03
bs_64x1           2.763946e-03
padding_0         2.307477e-03
Ldg               2.057492e-03
bs_64x4           1.644941e-03
bs_256x4          1.392080e-03
bs_512x2          1.274008e-03
bs_32x4           1.238470e-03
bs_128x2          1.082717e-03
bs_64x16          1.024975e-03
padding_192       1.018258e-03
bs_128x8          1.013524e-03
bs_32x32          9.287911e-04
padding_352       6.983680e-04
bs_1024x1         5.621562e-04
padding_

                  importance
bs_32x1             0.351674
LocalMemory         0.307093
pixelPerThread_1    0.088847
bs_1024x1           0.058541
pixelPerThread_2    0.038053
bs_512x1            0.014378
bs_512x2            0.013399
Array2D             0.010147
bs_256x4            0.009422
bs_32x32            0.008460
OpenCL              0.007794
bs_128x8            0.007208
CUDA                0.007179
bs_128x1            0.007013
bs_64x1             0.006901
bs_32x2             0.006573
bs_64x16            0.006532
pixelPerThread_4    0.004901
padding_512         0.004447
bs_256x2            0.004417
TextureMemory       0.004283
bs_32x8             0.003877
pixelPerThread_3    0.003782
bs_32x4             0.003046
Ldg                 0.002736
Linear1D            0.002727
padding_96          0.002192
bs_256x1            0.002176
Linear2D            0.002166
bs_128x4            0.002141
padding_0           0.001975
bs_64x2             0.000749
padding_384         0.000721
bs_32x16      